In [3]:
import binascii
import struct

def convert_hex_to_float(hex_str, endianess = "big"):
    # Convert the hexadecimal string to binary data (bytes)
    binary_data = binascii.unhexlify(hex_str)
    symbol = ">f"  # big-endian
    if endianess == 'little':
        symbol = "<f"  # little-endian
    float_value = struct.unpack(symbol, binary_data)[0]
    return float_value


# Define the hexadecimal string representing 4 bytes of a float in little-endian format
hex_str = '000001c5'
endianess = "big"
print(convert_hex_to_float(hex_str, endianess))  # Output: 100.0

6.347882043391421e-43


In [19]:
def float_to_hex(value: float, endian: str = 'little') -> str:
    """
    Convert a float to its hexadecimal representation.

    Args:
        value (float): The float number to convert.
        endian (str): 'little' for little-endian, 'big' for big-endian.

    Returns:
        str: Hexadecimal string representing the float.
    """
    # Choose format based on endianness
    if endian == 'little':
        fmt = '<f'  # Little-endian float
    elif endian == 'big':
        fmt = '>f'  # Big-endian float
    else:
        raise ValueError("Endian must be 'little' or 'big'")

    # Pack the float to 4-byte binary and convert to hex string
    binary_data = struct.pack(fmt, value)
    hex_str = binascii.hexlify(binary_data).decode('ascii')
    return hex_str

float_value = 660.780945
endianess = 'big'
print(float_to_hex(float_value, endianess))

442531fb


In [7]:
import struct
from typing import List, Tuple, Union, Any

STRUCT_FMT_MAP = {
    ('int', 2): ('<h', '>h'),
    ('int', 4): ('<i', '>i'),
    ('int', 8): ('<q', '>q'),
    ('uint', 2): ('<H', '>H'),
    ('uint', 4): ('<I', '>I'),
    ('uint', 8): ('<Q', '>Q'),
    ('float', 4): ('<f', '>f'),
    ('double', 8): ('<d', '>d'),
}

def try_parse_target(target_str: str) -> Tuple[Any, Any]:
    """
    Try to parse target string into int and float forms.
    Returns (int_value_or_None, float_value_or_None).
    """
    int_val = None
    float_val = None
    try:
        int_val = int(target_str)
    except ValueError:
        pass
    try:
        float_val = float(target_str)
    except ValueError:
        pass
    return int_val, float_val

def scan_hex_match_auto_type(
    hex_str: str,
    min_window_bytes: int,
    max_window_bytes: int,
    target_str: str,
    step_nibble: int = 1,
    float_tol: float = 1e-4
) -> List[Tuple[int, int, Union[float, int], str, str]]:
    """
    Scan hex string by sliding window with byte-aligned sizes and nibble steps.
    Automatically tries int/uint and float/double types of different sizes.
    Attempts to parse target_str into int and float and matches accordingly.

    Returns list of (offset_nibble, window_bytes, matched_value, dtype_str, endian_str)
    """
    results = []
    hex_len = len(hex_str)
    int_target, float_target = try_parse_target(target_str)

    # Candidate dtypes and sizes to try
    dtypes_to_try = [
        ('int', 2), ('int', 4), ('int', 8),
        ('uint', 2), ('uint', 4), ('uint', 8),
        ('float', 4), ('double', 8)
    ]

    for win_bytes in range(min_window_bytes, max_window_bytes + 1):
        win_nibbles = win_bytes * 2
        for offset in range(0, hex_len - win_nibbles + 1, step_nibble):
            sub_hex = hex_str[offset:offset + win_nibbles]

            try:
                byte_data = bytes.fromhex(sub_hex)
            except ValueError:
                continue

            for dtype_str, dtype_size in dtypes_to_try:
                if dtype_size != win_bytes:
                    continue

                if (dtype_str, dtype_size) not in STRUCT_FMT_MAP:
                    continue

                for endian_fmt, endian_str in zip(STRUCT_FMT_MAP[(dtype_str, dtype_size)], ['little', 'big']):
                    try:
                        val = struct.unpack(endian_fmt, byte_data)[0]
                    except struct.error:
                        continue

                    if dtype_str in ('float', 'double') and float_target is not None:
                        error = abs(val - float_target)
                        if error < float_tol:
                            results.append((offset, win_bytes, val, dtype_str, endian_str, error))
                    elif dtype_str in ('int', 'uint') and int_target is not None:
                        if val == int_target:
                            results.append((offset, win_bytes, val, dtype_str, endian_str, 0.0))

    return results


hex_data = '00000001c58e6d4400000000'
target_number_str = "6.347882043391421e-43"

matches = scan_hex_match_auto_type(
    hex_str=hex_data,
    min_window_bytes=2,
    max_window_bytes=8,
    target_str=target_number_str,
    step_nibble=1,
    float_tol=1e-30
)

for match in matches:
    print(f"Offset (nibble): {match[0]}, Window (bytes): {match[1]}, Value: {match[2]}, Type: {match[3]}, Endian: {match[4]}, Error: {match[5]}")


Offset (nibble): 0, Window (bytes): 4, Value: 2.350988701644575e-38, Type: float, Endian: little, Error: 2.350925222824141e-38
Offset (nibble): 0, Window (bytes): 4, Value: 1.401298464324817e-45, Type: float, Endian: big, Error: 6.333869058748173e-43
Offset (nibble): 1, Window (bytes): 4, Value: 3.923635700109488e-44, Type: float, Endian: big, Error: 5.9555184733804726e-43
Offset (nibble): 2, Window (bytes): 4, Value: 6.347882043391421e-43, Type: float, Endian: big, Error: 0.0
Offset (nibble): 3, Window (bytes): 4, Value: 1.0167821657140873e-41, Type: float, Endian: big, Error: 9.53303345280173e-42
Offset (nibble): 4, Window (bytes): 4, Value: 1.627047646927545e-40, Type: float, Endian: big, Error: 1.6206997648841537e-40
Offset (nibble): 5, Window (bytes): 4, Value: 2.603284642874858e-39, Type: float, Endian: big, Error: 2.602649854670519e-39
Offset (nibble): 6, Window (bytes): 4, Value: 7.257074194572806e-38, Type: float, Endian: big, Error: 7.257010715752372e-38
Offset (nibble): 10, 